In [1]:
import pandas as pd

import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json 
import seaborn as sns 

In [2]:
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "axes.titlesize": 24,        # Larger axes/title fonts
        "legend.fontsize": 24,
    }
)
sns.set_context("talk")

In [3]:
palette_name = 'ocean_sunset'
palette = json.load(open(f'../palettes/{palette_name}.json', 'r'))

## SipIT

In [4]:
all_dfs = []

for file in os.listdir('../data/sipit_correctness/'):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join('../data/sipit_correctness/', file))
        all_dfs.append(df)
df_sipit = pd.concat(all_dfs, ignore_index=True)

In [5]:
df_sipit

,dataset,index,layer,learning_rate,token_length,match,inversion_time,timesteps,times
0,0000_wikipedia_tokenized.json,0,12,1.0,20,True,14.141456,31_11_141_81_24_8_7_21_12_15_31_41_54_19_11_6_...,0.84_0.22_2.83_1.64_0.48_0.16_0.14_0.42_0.25_0...
1,0000_wikipedia_tokenized.json,1,12,1.0,20,True,27.672702,580_20_8_18_30_10_18_230_28_9_35_11_34_48_17_1...,11.07_0.40_0.17_0.35_0.60_0.20_0.37_4.72_0.57_...
2,0000_wikipedia_tokenized.json,2,12,1.0,20,True,35.754205,934_27_19_32_60_42_18_132_12_13_38_18_96_64_15...,17.97_0.56_0.39_0.67_1.25_0.89_0.38_2.68_0.24_...
3,0000_wikipedia_tokenized.json,3,12,1.0,20,True,46.990014,148_17_16_21_13_19_10_34_1554_28_48_52_40_13_8...,2.88_0.35_0.32_0.43_0.26_0.39_0.21_0.71_31.87_...
4,0000_wikipedia_tokenized.json,4,12,1.0,20,True,41.099558,362_25_20_14_39_30_14_37_41_18_84_21_1101_25_1...,7.00_0.51_0.40_0.28_0.81_0.60_0.27_0.75_0.85_0...
...,...,...,...,...,...,...,...,...,...
995,0015_random-0015_random.json,9,12,1.0,120,True,81.000368,96_37_17_22_25_12_16_26_24_29_24_15_33_10_16_1...,1.84_0.76_0.35_0.45_0.52_0.24_0.32_0.53_0.51_0...
996,0016_random-0016_random.json,9,12,1.0,140,True,96.429771,50_13_20_33_25_17_12_40_17_22_18_34_15_25_23_9...,0.96_0.27_0.41_0.69_0.51_0.35_0.24_0.83_0.34_0...
997,0017_random-0017_random.json,9,12,1.0,160,True,227.986446,3634_14_16_22_13_34_31_20_32_20_18_15_9_28_30_...,69.75_0.28_0.33_0.45_0.28_0.70_0.64_0.41_0.66_...
998,0018_random-0018_random.json,9,12,1.0,180,True,201.287705,44_10_16_39_20_14_18_187_33_28_23_18_15_23_38_...,2.96_0.71_1.00_2.61_5.38_0.84_0.38_4.01_0.71_0...


In [7]:
df_sipit.drop(columns=['learning_rate', 'layer'], inplace=True)

## Exhaustive

In [9]:
all_dfs = []

for file in os.listdir('../data/sipit_exhaustive/'):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join('../data/sipit_exhaustive/', file))
        all_dfs.append(df)
df_brute = pd.concat(all_dfs, ignore_index=True)

In [14]:
df1_reset = df_sipit.reset_index().rename(columns={"index": "sipit_index"})
df2_reset = df_brute.reset_index().rename(columns={"index": "brute_index"})

merged = df1_reset.merge(
    df2_reset,
    left_on=["sipit_index", "dataset"],
    right_on=["brute_index", "dataset"],
    how="inner",
    suffixes=("_sipit", "_brute")
)


In [19]:
merged.head()

,level_0_sipit,dataset,sipit_index,token_length_sipit,match_sipit,inversion_time_sipit,timesteps_sipit,times_sipit,level_0_brute,brute_index,layer,learning_rate,token_length_brute,match_brute,inversion_time_brute,timesteps_brute,times_brute
0,0,0000_wikipedia_tokenized.json,0,20,True,14.141456,31_11_141_81_24_8_7_21_12_15_31_41_54_19_11_6_...,0.84_0.22_2.83_1.64_0.48_0.16_0.14_0.42_0.25_0...,0,0,12,1.0,20,True,3856.145249,8999_6465_13892_33737_10505_24017_48379_30390_...,69.64_50.72_109.39_276.40_82.98_188.67_381.64_...
1,1,0000_wikipedia_tokenized.json,1,20,True,27.672702,580_20_8_18_30_10_18_230_28_9_35_11_34_48_17_1...,11.07_0.40_0.17_0.35_0.60_0.20_0.37_4.72_0.57_...,1,1,12,1.0,20,True,3742.685253,37649_12204_31431_15088_45402_12571_10708_9485...,295.50_96.05_248.22_119.97_360.52_101.30_86.37...
2,2,0000_wikipedia_tokenized.json,2,20,True,35.754205,934_27_19_32_60_42_18_132_12_13_38_18_96_64_15...,17.97_0.56_0.39_0.67_1.25_0.89_0.38_2.68_0.24_...,2,2,12,1.0,20,True,4147.350310,45682_41251_2701_33952_28545_35799_1704_17909_...,357.13_323.99_21.29_268.35_226.31_284.45_13.63...
3,3,0000_wikipedia_tokenized.json,3,20,True,46.990014,148_17_16_21_13_19_10_34_1554_28_48_52_40_13_8...,2.88_0.35_0.32_0.43_0.26_0.39_0.21_0.71_31.87_...,3,3,12,1.0,20,True,3199.874364,32850_11079_40223_9442_27301_30922_2739_35463_...,262.50_88.07_323.34_75.15_217.93_247.04_21.64_...
4,4,0000_wikipedia_tokenized.json,4,20,True,41.099558,362_25_20_14_39_30_14_37_41_18_84_21_1101_25_1...,7.00_0.51_0.40_0.28_0.81_0.60_0.27_0.75_0.85_0...,4,4,12,1.0,20,True,4335.024322,34146_20627_1017_25867_43374_22209_36394_35463...,276.19_167.54_8.23_207.68_346.07_176.95_288.42...


In [22]:
for method in ['sipit', 'brute']:

    avg_inversion_time = merged[f'inversion_time_{method}'].mean()
    std_inversion_time = merged[f'inversion_time_{method}'].std()

    print(f"{method} inversion time: {avg_inversion_time:.2f} ± {std_inversion_time:.2f} seconds")


sipit inversion time: 28.01 ± 35.87 seconds
brute inversion time: 3889.61 ± 691.17 seconds
